# Task2

## Setup & Load

In [1]:
# Setup & load
from pathlib import Path
import pandas as pd
import numpy as np

# Define project paths
project_root      = Path.cwd().parent
TASK1_CLEAN_DIR   = project_root / "reports" / "task1" / "cleaned"
TASK2_DIR         = project_root / "reports" / "task2"
TASK2_DIR.mkdir(parents=True, exist_ok=True)

# Define file paths
csv_paths = {
    "dh_orig":   TASK1_CLEAN_DIR / "drillhole_original_clean.csv",
    "dh_dnn":    TASK1_CLEAN_DIR / "drillhole_dnn_clean.csv",
    "surf_orig": TASK1_CLEAN_DIR / "surface_original_clean.csv",
    "surf_dnn":  TASK1_CLEAN_DIR / "surface_dnn_clean.csv",
}

# Load CSVs
dfs = {k: pd.read_csv(v) for k, v in csv_paths.items()}

df_dh_orig_clean   = dfs["dh_orig"]
df_dh_dl_clean     = dfs["dh_dnn"]
df_surf_orig_clean = dfs["surf_orig"]
df_surf_dl_clean   = dfs["surf_dnn"]

# Print basic info
for k, df in dfs.items():
    print(f"{k:10s}: shape={df.shape}")

dh_orig   : shape=(2466130, 9)
dh_dnn    : shape=(920225, 8)
surf_orig : shape=(981318, 6)
surf_dnn  : shape=(544886, 5)


### Summarize Latitude/Longitude Extent

In [2]:
# Latitude/Longitude coverage summary

def extent_for(df: pd.DataFrame, lat_col: str, lon_col: str) -> dict:
    lat = pd.to_numeric(df[lat_col], errors="coerce").dropna()
    lon = pd.to_numeric(df[lon_col], errors="coerce").dropna()
    return {
        "LatMin": lat.min() if not lat.empty else None,
        "LatMax": lat.max() if not lat.empty else None,
        "LonMin": lon.min() if not lon.empty else None,
        "LonMax": lon.max() if not lon.empty else None,
        "Count": int(df.shape[0])
    }

rows = []
datasets = {
    "Drillhole Original": (df_dh_orig_clean, "LATITUDE", "LONGITUDE"),
    "Drillhole DNN":      (df_dh_dl_clean,   "LATITUDE", "LONGITUDE"),
    "Surface Original":   (df_surf_orig_clean, "DLAT", "DLONG"),
    "Surface DNN":        (df_surf_dl_clean,   "DLAT", "DLONG"),
}

for name, (df, lat_col, lon_col) in datasets.items():
    if {lat_col, lon_col}.issubset(df.columns):
        e = extent_for(df, lat_col, lon_col)
        e["Dataset"] = name
        rows.append(e)

extents_df = pd.DataFrame(rows)[["Dataset","Count","LatMin","LatMax","LonMin","LonMax"]]
display(extents_df)

,Dataset,Count,LatMin,LatMax,LonMin,LonMax
0,Drillhole Original,2466130,-34.945354,-27.025372,114.675570,122.131294
1,Drillhole DNN,920225,-34.945354,-27.025372,114.676380,122.107080
2,Surface Original,981318,-34.990154,-26.915041,114.519930,122.135070
3,Surface DNN,544886,-34.927277,-27.102000,114.547104,122.122314


## Samples

A subset for testing (here I set up the same region as in the paper).
Can either use this smaller dataset to focus only on the study area, or use the full cleaned datasets for broader analysis. 

In [3]:
# Define a bounding box (latitude/longitude region)
lat_min, lat_max = -30.0, -27.5
lon_min, lon_max = 120.0, 121.5

def filter_by_bbox(df: pd.DataFrame, lat_col: str, lon_col: str) -> pd.DataFrame:
    lat = pd.to_numeric(df[lat_col], errors="coerce")
    lon = pd.to_numeric(df[lon_col], errors="coerce")
    mask = (
        lat.notna() & lon.notna() &
        (lat >= lat_min) & (lat <= lat_max) &
        (lon >= lon_min) & (lon <= lon_max)
    )
    return df.loc[mask].copy()

# Use relative path (like in Task1)
SAMPLE_DIR = Path("../reports/task2/sample_csv")
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

# Filter and export
samples = {}
for name, (df, lat_col, lon_col) in datasets.items():
    if {lat_col, lon_col}.issubset(df.columns):
        sub = filter_by_bbox(df, lat_col, lon_col)
        samples[name] = sub
        out_path = SAMPLE_DIR / f"{name.lower().replace(' ', '_')}_sample.csv"
        sub.to_csv(out_path, index=False)
        print(f"[SAVE] {name:18s} -> rows={sub.shape[0]}")

print("Saved Task2 sample datasets to:", SAMPLE_DIR)

[SAVE] Drillhole Original -> rows=403
[SAVE] Drillhole DNN      -> rows=294
[SAVE] Surface Original   -> rows=210
[SAVE] Surface DNN        -> rows=531
Saved Task2 sample datasets to: ../reports/task2/sample_csv


## Test 1

## Drillhole Comparison: ORIG vs DL

This notebook processes two drillhole datasets:
- **ORIG**: original assay values  
- **DL**: values predicted by the deep learning model  

We align them on both:
1. Longitude/latitude grid cells  
2. Overlapping depth intervals  

**Diff logic**
diff = DL - ORIG (default)  

**Outputs**
1. `*_overlaps.csv`: detailed overlapping intervals  
2. `*_points.csv`: representative points (depth midpoints), used for 3D visualization in Streamlit.

### Envrionment & Parameters

In [4]:
# Input: cleaned DataFrames 
# - df_dh_orig_clean
# - df_dh_dl_clean

# Output directory (relative to project root)
OUT_DIR = Path("../reports/task2")
BASE_NAME = "drillhole"

# Key parameters
GRID_STEP_DEG = 1e-4   # grid size in degrees (~10 m)
Z_STEP_M      = 1.0    # depth bin size in meters
DIFF_MODE     = "dl_minus_orig"  # or "orig_minus_dl"

In [5]:
# Helper Functions 
def to_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def snap_xy_to_grid(df: pd.DataFrame,
                    lon_col="LONGITUDE",
                    lat_col="LATITUDE",
                    step=1e-4) -> pd.DataFrame:
    """Snap longitude/latitude to regular grid."""
    df = df.copy()
    df["_gx"] = np.floor(df[lon_col] / step).astype(np.int64)
    df["_gy"] = np.floor(df[lat_col] / step).astype(np.int64)
    return df

In [6]:
# Align by XY & Depth Interval
def align_by_xy_and_depth_interval(
    df_orig, df_dl,
    lon_col="LONGITUDE", lat_col="LATITUDE",
    from_col="FROMDEPTH", to_col="TODEPTH",
    cu_col="Cu_ppm",
    grid_step=1e-4,
    diff_mode="dl_minus_orig"
) -> pd.DataFrame:
    """
    Align ORIG and DL by (lon,lat) grid and overlapping depth intervals.
    Returns overlaps with both Cu values and diff.
    """
    need = [lon_col, lat_col, from_col, to_col, cu_col]
    A = to_numeric(df_orig[need].dropna(), need)
    B = to_numeric(df_dl  [need].dropna(), need)

    A = A.loc[A[from_col] < A[to_col]].copy()
    B = B.loc[B[from_col] < B[to_col]].copy()

    A = snap_xy_to_grid(A, lon_col, lat_col, step=grid_step)
    B = snap_xy_to_grid(B, lon_col, lat_col, step=grid_step)

    parts = []
    for (gx, gy), gA in A.groupby(["_gx", "_gy"]):
        gB = B[(B["_gx"] == gx) & (B["_gy"] == gy)]
        if gB.empty: continue

        gA = gA.rename(columns={from_col:"fromA", to_col:"toA", cu_col:"Cu_orig",
                                lon_col:"lonA", lat_col:"latA"})
        gB = gB.rename(columns={from_col:"fromB", to_col:"toB", cu_col:"Cu_dl",
                                lon_col:"lonB", lat_col:"latB"})
        gA["__k"] = 1; gB["__k"] = 1
        M = gA.merge(gB, on="__k").drop(columns="__k")

        M = M.loc[(M["fromA"] < M["toB"]) & (M["fromB"] < M["toA"])].copy()
        if M.empty: continue

        M["from"] = M[["fromA","fromB"]].max(axis=1)
        M["to"]   = M[["toA","toB"]].min(axis=1)
        M = M.loc[M["from"] < M["to"]].copy()
        if M.empty: continue

        if diff_mode == "dl_minus_orig":
            M["diff"] = M["Cu_dl"] - M["Cu_orig"]
        else:
            M["diff"] = M["Cu_orig"] - M["Cu_dl"]

        M["_gx"] = gx; M["_gy"] = gy
        parts.append(M[["from","to","Cu_orig","Cu_dl","diff",
                        "lonA","latA","lonB","latB","_gx","_gy"]])

    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()

In [7]:
# Convert Overlaps to Points
def overlaps_to_points(aligned, grid_step=1e-4, z_step=1.0, agg="mean"):
    """
    Convert overlap intervals into representative 3D points.
    DEPTH = midpoint; LONG/LAT = average of both sides.
    """
    if aligned is None or len(aligned) == 0:
        return pd.DataFrame(columns=["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT"])

    df = aligned.copy()
    df["DEPTH"] = (df["from"] + df["to"]) / 2.0
    df["LONGITUDE"] = (df["lonA"] + df["lonB"]) / 2.0
    df["LATITUDE"]  = (df["latA"] + df["latB"]) / 2.0
    df = df.rename(columns={"Cu_orig":"CU_ORIG","Cu_dl":"CU_DL","diff":"DIFF", "diff_pct":"DIFF_PCT"})

    df["_gx"] = np.floor(df["LONGITUDE"]/grid_step).astype(np.int64)
    df["_gy"] = np.floor(df["LATITUDE"]/grid_step).astype(np.int64)
    df["_gz"] = np.floor(df["DEPTH"]/z_step).astype(np.int64)

    grouped = (
        df.groupby(["_gx","_gy","_gz"], as_index=False)
          .agg({"LONGITUDE":agg,"LATITUDE":agg,"DEPTH":agg,
                "CU_ORIG":agg,"CU_DL":agg,"DIFF":agg, "DIFF_PCT":  agg})
    )
    return grouped[[ "LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT"]]

In [8]:
# Pipeline Function
def process_drillhole_pair_csv(
    df_orig, df_dl,
    out_dir="reports/task2",
    base_name="drillhole",
    grid_step=1e-4,
    z_step=1.0,
    diff_mode="dl_minus_orig"
):
    """Run pipeline and save CSVs."""
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    aligned = align_by_xy_and_depth_interval(
        df_orig, df_dl, grid_step=grid_step, diff_mode=diff_mode
    )
    print(f"[INFO] overlaps: {len(aligned)}")

    eps = 1e-9
    denom = aligned["Cu_dl"].where(aligned["Cu_dl"].abs() >= eps, np.nan)
    aligned["diff_pct"] = 100.0 * aligned["diff"] / denom

    points = overlaps_to_points(aligned, grid_step=grid_step, z_step=z_step, agg="mean")
    print(f"[INFO] points: {len(points)}")

    overlaps_csv = Path(out_dir) / f"{base_name}_overlaps.csv"
    points_csv   = Path(out_dir) / f"{base_name}_points.csv"
    aligned.to_csv(overlaps_csv, index=False)
    points.to_csv(points_csv, index=False)

    print(f"[SAVE] {overlaps_csv}")
    print(f"[SAVE] {points_csv}")
    return str(overlaps_csv), str(points_csv)

In [9]:
overlap_path, points_path = process_drillhole_pair_csv(
    df_dh_orig_clean, df_dh_dl_clean,
    out_dir=OUT_DIR,
    base_name=BASE_NAME,
    grid_step=GRID_STEP_DEG,
    z_step=Z_STEP_M,
    diff_mode=DIFF_MODE
)

[INFO] overlaps: 260797
[INFO] points: 90439
[SAVE] ../reports/task2/drillhole_overlaps.csv
[SAVE] ../reports/task2/drillhole_points.csv


**Method**

**Drillhole Diff Method**
1. Grid snapping (XY):

Longitude & latitude are snapped to grid cells of size grid_step.

2.Depth intervals (Z):
Each record is an interval [FROM, TO] rather than a single depth.

3. Overlap rule:
Two intervals are compared only if they overlap in the same XY cell.

Intersection only:
If [fromA,toA] and [fromB,toB] overlap, keep the intersecting part.
- Diff calculation:
	diff = Cu_dl – Cu_orig (or the reverse, depending on mode).

Outputs:
- Overlap intervals with both Cu values and diff
- Point cloud representation (midpoints of overlaps) for 3D mapping

In [10]:
df_overlaps = pd.read_csv("../reports/task2/drillhole_overlaps.csv")
df_points   = pd.read_csv("../reports/task2/drillhole_points.csv")

display(df_overlaps.head())
display(df_points.head())

,from,to,Cu_orig,Cu_dl,diff,lonA,latA,lonB,latB,_gx,_gy,diff_pct
0,133.0,134.0,80.0,9.790,-70.210,114.67665,-27.768700,114.67665,-27.768700,1146766,-277687,-717.160368
1,134.0,135.0,55.0,18.963,-36.037,114.67665,-27.768700,114.67665,-27.768700,1146766,-277687,-190.038496
2,45.0,46.0,74.0,26.771,-47.229,115.41368,-27.418915,115.41368,-27.418915,1154136,-274190,-176.418513
3,45.0,46.0,74.0,26.771,-47.229,115.41368,-27.418915,115.41368,-27.418915,1154136,-274190,-176.418513
4,39.0,40.0,90.0,16.608,-73.392,115.41368,-27.418915,115.41368,-27.418915,1154136,-274190,-441.907514


,LONGITUDE,LATITUDE,DEPTH,CU_ORIG,CU_DL,DIFF,DIFF_PCT
0,114.67665,-27.768700,133.5,80.0,9.790,-70.210,-717.160368
1,114.67665,-27.768700,134.5,55.0,18.963,-36.037,-190.038496
2,115.41368,-27.418915,31.5,243.0,59.017,-183.983,-311.745768
3,115.41368,-27.418915,39.5,90.0,16.608,-73.392,-441.907514
4,115.41368,-27.418915,45.5,74.0,26.771,-47.229,-176.418513


## Test 2

Imports & config

In [11]:
# --- Imports & global config ---
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple, Dict, List

# Output directory: put everything under task2/difference
OUT_DIR = Path("../reports/task2/difference")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Grid parameters
GRID_STEP_DEG = 1e-4   # XY grid step in degrees (~10 m)
Z_STEP_M      = 1.0    # Z bin size in meters

# DIFF sign convention
DIFF_MODE = "dl_minus_orig"   # or "orig_minus_dl"

Standardize helpers

In [12]:
# --- Standardize column names for both data types ---

def standardize_drillhole(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure drillhole dataframe has consistent columns:
    LON, LAT, FROM, TO, CU (+ keep anything else).
    """
    rename_map = {
        "LONGITUDE": "LON",
        "LATITUDE":  "LAT",
        "FROMDEPTH": "FROM",
        "TODEPTH":   "TO",
        "Cu_ppm":    "CU"
    }
    out = df.rename(columns=rename_map).copy()
    for c in ["LON", "LAT", "FROM", "TO", "CU"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=["LON", "LAT", "FROM", "TO", "CU"])
    out = out.loc[out["FROM"] < out["TO"]]
    return out

def standardize_surface(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure surface dataframe has consistent columns:
    LON, LAT, CU (no depth).
    Accepts DLAT/DLONG or LATITUDE/LONGITUDE.
    """
    rename_map = {
        "DLAT":      "LAT",
        "DLONG":     "LON",
        "LATITUDE":  "LAT",
        "LONGITUDE": "LON",
        "Cu_ppm":    "CU",
    }
    out = df.rename(columns=rename_map).copy()
    for c in ["LON", "LAT", "CU"]:
        out[c] = pd.to_numeric(out[c], errors="coerce")
    out = out.dropna(subset=["LON", "LAT", "CU"])
    return out

# --- XY grid helpers used by both drillhole & surface ---

def snap_xy_to_grid(df: pd.DataFrame, step: float = GRID_STEP_DEG) -> pd.DataFrame:
    """Add integer XY grid indices: _gx, _gy."""
    out = df.copy()
    out["_gx"] = np.floor(out["LON"] / step).astype(np.int64)
    out["_gy"] = np.floor(out["LAT"] / step).astype(np.int64)
    return out

def grid_cell_centers(gx: np.ndarray, gy: np.ndarray, step: float = GRID_STEP_DEG) -> Tuple[np.ndarray, np.ndarray]:
    """Convert integer grid indices (_gx, _gy) to cell-center lon/lat."""
    lon = (gx + 0.5) * step
    lat = (gy + 0.5) * step
    return lon, lat

def count_xy_cells(df: pd.DataFrame) -> int:
    """Number of unique XY grid cells in a frame that already has _gx/_gy."""
    if ("_gx" not in df.columns) or ("_gy" not in df.columns):
        return 0
    return int(df.drop_duplicates(["_gx", "_gy"]).shape[0])

 Drillhole split（overlap / orig-only / dl-only）

In [13]:
# --- Drillhole splitting by XY cell and overlapping depth intervals ---

def split_drillhole_overlap(
    df_orig: pd.DataFrame,
    df_dl: pd.DataFrame,
    grid_step: float = GRID_STEP_DEG,
    diff_mode: str = DIFF_MODE,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Return (overlap_df, orig_only_df, dl_only_df) for drillhole:
    - Overlap requires same XY grid cell AND overlapping depth intervals.
    - orig-only: ORIG intervals with no DL overlap in the same XY cell.
    - dl-only:   DL intervals with no ORIG overlap in the same XY cell.
    """
    A = snap_xy_to_grid(df_orig, step=grid_step).reset_index(drop=False).rename(columns={"index":"_idxA"})
    B = snap_xy_to_grid(df_dl,   step=grid_step).reset_index(drop=False).rename(columns={"index":"_idxB"})

    overlaps: List[pd.DataFrame] = []
    orig_only_rows: List[pd.DataFrame] = []
    dl_only_rows: List[pd.DataFrame] = []

    for (gx, gy), gA in A.groupby(["_gx", "_gy"]):
        gB = B[(B["_gx"] == gx) & (B["_gy"] == gy)]
        if gB.empty:
            orig_only_rows.append(gA)
            continue

        a = gA.rename(columns={"FROM":"fromA", "TO":"toA", "CU":"Cu_orig", "LON":"lonA", "LAT":"latA"})
        b = gB.rename(columns={"FROM":"fromB", "TO":"toB", "CU":"Cu_dl",   "LON":"lonB", "LAT":"latB"})
        a["__k"] = 1; b["__k"] = 1
        M = a.merge(b, on="__k", how="inner").drop(columns="__k")

        if M.empty:
            orig_only_rows.append(gA)
            continue

        M = M.loc[(M["fromA"] < M["toB"]) & (M["fromB"] < M["toA"])].copy()
        if M.empty:
            orig_only_rows.append(gA)
            continue

        M["from"] = M[["fromA", "fromB"]].max(axis=1)
        M["to"]   = M[["toA",   "toB"]].min(axis=1)
        M = M.loc[M["from"] < M["to"]].copy()
        if M.empty:
            orig_only_rows.append(gA)
            continue

        if diff_mode == "dl_minus_orig":
            M["diff"] = M["Cu_dl"] - M["Cu_orig"]
        else:
            M["diff"] = M["Cu_orig"] - M["Cu_dl"]

        eps = 1e-9
        denom = M["Cu_dl"].where(M["Cu_dl"].abs() >= eps, np.nan)
        M["diff_pct"] = 100.0 * M["diff"] / denom

        M["_gx"] = gx; M["_gy"] = gy
        overlaps.append(M[[
            "from","to","Cu_orig","Cu_dl","diff","diff_pct",
            "lonA","latA","lonB","latB","_gx","_gy","_idxA","_idxB"
        ]])

        usedA = set(M["_idxA"].unique())
        usedB = set(M["_idxB"].unique())
        if len(usedA) < len(gA):
            orig_only_rows.append(gA.loc[~gA["_idxA"].isin(usedA)])
        if len(usedB) < len(gB):
            dl_only_rows.append(gB.loc[~gB["_idxB"].isin(usedB)])

    overlap_df = pd.concat(overlaps, ignore_index=True) if overlaps else pd.DataFrame(
        columns=["from","to","Cu_orig","Cu_dl","diff","diff_pct","lonA","latA","lonB","latB","_gx","_gy","_idxA","_idxB"]
    )
    orig_only_df = pd.concat(orig_only_rows, ignore_index=True) if orig_only_rows else pd.DataFrame(columns=A.columns)
    dl_only_df   = pd.concat(dl_only_rows,   ignore_index=True) if dl_only_rows   else pd.DataFrame(columns=B.columns)

    return overlap_df, orig_only_df, dl_only_df

Intervals → 3D points/voxels

In [14]:
# --- Convert drillhole intervals to representative 3D points (for Streamlit) ---

def dh_intervals_to_points(
    overlap_df: pd.DataFrame,
    orig_only_df: pd.DataFrame,
    dl_only_df: pd.DataFrame,
    grid_step: float = GRID_STEP_DEG,
    z_step: float = Z_STEP_M,
    agg: str = "mean",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, Dict[str, int]]:
    """
    Build 3D points for three sources (overlap / orig-only / dl-only) and an 'all' union.
    Each point: LONGITUDE, LATITUDE, DEPTH, CU_ORIG, CU_DL, DIFF, DIFF_PCT, SOURCE
    Returns (pts_overlap, pts_origonly, pts_dlonly, pts_all, counts_dict)
    """
    def _bin_and_agg(df_pts: pd.DataFrame) -> pd.DataFrame:
        if df_pts.empty:
            return df_pts
        df = df_pts.copy()
        df["_gx"] = np.floor(df["LONGITUDE"]/grid_step).astype(np.int64)
        df["_gy"] = np.floor(df["LATITUDE"] /grid_step).astype(np.int64)
        df["_gz"] = np.floor(df["DEPTH"]   /z_step   ).astype(np.int64)
        grouped = (
            df.groupby(["_gx","_gy","_gz","SOURCE"], as_index=False)
              .agg({
                  "LONGITUDE":agg, "LATITUDE":agg, "DEPTH":agg,
                  "CU_ORIG":agg, "CU_DL":agg, "DIFF":agg, "DIFF_PCT":agg
              })
        )
        return grouped[["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT","SOURCE","_gx","_gy","_gz"]]

    pts_overlap = pd.DataFrame(columns=["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT","SOURCE"])
    if not overlap_df.empty:
        O = overlap_df.copy()
        O["DEPTH"]     = (O["from"] + O["to"]) / 2.0
        O["LONGITUDE"] = (O["lonA"] + O["lonB"]) / 2.0
        O["LATITUDE"]  = (O["latA"] + O["latB"]) / 2.0
        O = O.rename(columns={"Cu_orig":"CU_ORIG","Cu_dl":"CU_DL"})
        O["SOURCE"] = "overlap"
        pts_overlap = O[["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","diff","diff_pct","SOURCE"]].rename(
            columns={"diff":"DIFF","diff_pct":"DIFF_PCT"}
        )

    pts_origonly = pd.DataFrame(columns=pts_overlap.columns)
    if not orig_only_df.empty:
        A = orig_only_df.copy()
        A["DEPTH"]     = (A["FROM"] + A["TO"]) / 2.0
        A["LONGITUDE"] = A["LON"]; A["LATITUDE"] = A["LAT"]
        A["CU_ORIG"]   = A["CU"];  A["CU_DL"]    = np.nan
        A["DIFF"]      = np.nan;   A["DIFF_PCT"] = np.nan
        A["SOURCE"]    = "orig_only"
        pts_origonly = A[["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT","SOURCE"]]

    pts_dlonly = pd.DataFrame(columns=pts_overlap.columns)
    if not dl_only_df.empty:
        B = dl_only_df.copy()
        B["DEPTH"]     = (B["FROM"] + B["TO"]) / 2.0
        B["LONGITUDE"] = B["LON"]; B["LATITUDE"] = B["LAT"]
        B["CU_ORIG"]   = np.nan;   B["CU_DL"]    = B["CU"]
        B["DIFF"]      = np.nan;   B["DIFF_PCT"] = np.nan
        B["SOURCE"]    = "dl_only"
        pts_dlonly = B[["LONGITUDE","LATITUDE","DEPTH","CU_ORIG","CU_DL","DIFF","DIFF_PCT","SOURCE"]]

    vox_overlap  = _bin_and_agg(pts_overlap)
    vox_origonly = _bin_and_agg(pts_origonly)
    vox_dlonly   = _bin_and_agg(pts_dlonly)
    vox_all      = pd.concat([vox_overlap, vox_origonly, vox_dlonly], ignore_index=True)

    counts = {
        # raw counts (approximate back from indices present in splits)
        "raw_orig_rows": int(orig_only_df.shape[0] + overlap_df["_idxA"].nunique() if not overlap_df.empty else orig_only_df.shape[0]),
        "raw_dl_rows":   int(dl_only_df.shape[0]   + overlap_df["_idxB"].nunique() if not overlap_df.empty else dl_only_df.shape[0]),
        "vox_overlap":   int(vox_overlap.drop_duplicates(["_gx","_gy","_gz"]).shape[0]),
        "vox_orig_only": int(vox_origonly.drop_duplicates(["_gx","_gy","_gz"]).shape[0]),
        "vox_dl_only":   int(vox_dlonly.drop_duplicates(["_gx","_gy","_gz"]).shape[0]),
        "vox_all":       int(vox_all.drop_duplicates(["_gx","_gy","_gz"]).shape[0]),
    }
    return vox_overlap, vox_origonly, vox_dlonly, vox_all, counts

Run drillhole pipeline end-to-end

In [15]:
# --- Run drillhole pipeline end-to-end ---

# 1) Standardize
dh_orig = standardize_drillhole(df_dh_orig_clean)
dh_dl   = standardize_drillhole(df_dh_dl_clean)
print("Drillhole standardized shapes:", dh_orig.shape, dh_dl.shape)

# 2) Split by overlap
dh_overlap, dh_orig_only, dh_dl_only = split_drillhole_overlap(dh_orig, dh_dl)
print("[Drillhole] raw rows:",
      f"overlap pairs={len(dh_overlap):,}, orig-only={len(dh_orig_only):,}, dl-only={len(dh_dl_only):,}")

# 3) Convert intervals -> 3D voxel points
pts_overlap, pts_origonly, pts_dlonly, pts_all, dh_counts = dh_intervals_to_points(
    dh_overlap, dh_orig_only, dh_dl_only
)
print("[Drillhole] voxel counts:",
      f"overlap={dh_counts['vox_overlap']:,}, orig-only={dh_counts['vox_orig_only']:,},",
      f"dl-only={dh_counts['vox_dl_only']:,}, all={dh_counts['vox_all']:,}")

# 4) Export CSVs for Streamlit (3D viewer)
pts_overlap.to_csv(OUT_DIR / "drillhole_points_overlap.csv", index=False)
pts_origonly.to_csv(OUT_DIR / "drillhole_points_origonly.csv", index=False)
pts_dlonly  .to_csv(OUT_DIR / "drillhole_points_dlonly.csv", index=False)
pts_all     .to_csv(OUT_DIR / "drillhole_points_all.csv", index=False)

# Also keep interval-level overlap table (useful for audit)
dh_overlap.to_csv(OUT_DIR / "drillhole_overlaps.csv", index=False)

print("[SAVE] Drillhole CSVs written to", OUT_DIR)

Drillhole standardized shapes: (2464631, 9) (920013, 8)
[Drillhole] raw rows: overlap pairs=260,797, orig-only=2,317,904, dl-only=109,124
[Drillhole] voxel counts: overlap=90,439, orig-only=1,787,661, dl-only=50,084, all=1,926,606
[SAVE] Drillhole CSVs written to ../reports/task2/difference


Summary for drillhole

In [16]:
def pretty_pct(n, d):
    return "-" if d == 0 else f"{100.0*n/d:.2f}%"

# 1) Raw rows
rows_raw = [
    ["Raw rows", "ORIG", raw_orig, "-"],
    ["Raw rows", "DL", raw_dl, "-"],
]

# 2) XY cells
rows_xy = [
    ["XY grid", "ORIG", xy_orig, pretty_pct(xy_orig, raw_orig)],
    ["XY grid", "DL", xy_dl, pretty_pct(xy_dl, raw_dl)],
]

# 3) Interval split
total_interval_rows = len(dh_overlap) + len(dh_orig_only) + len(dh_dl_only)
rows_interval = [
    ["Interval split", "Overlap",   len(dh_overlap),   pretty_pct(len(dh_overlap), total_interval_rows)],
    ["Interval split", "Orig-only", len(dh_orig_only), pretty_pct(len(dh_orig_only), total_interval_rows)],
    ["Interval split", "DL-only",   len(dh_dl_only),   pretty_pct(len(dh_dl_only), total_interval_rows)],
    ["Interval split", "TOTAL",     total_interval_rows, "100.0%"],
]

# 4) Voxel split
vox_overlap  = dh_counts["vox_overlap"]
vox_origonly = dh_counts["vox_orig_only"]
vox_dlonly   = dh_counts["vox_dl_only"]
vox_all      = dh_counts["vox_all"]

rows_voxel = [
    ["Voxel split", "Overlap",   vox_overlap,   pretty_pct(vox_overlap, vox_all)],
    ["Voxel split", "Orig-only", vox_origonly, pretty_pct(vox_origonly, vox_all)],
    ["Voxel split", "DL-only",   vox_dlonly,   pretty_pct(vox_dlonly, vox_all)],
    ["Voxel split", "TOTAL",     vox_all, "100.0%"],
]

# Combine all into one DataFrame
tbl_all = pd.DataFrame(rows_raw + rows_xy + rows_interval + rows_voxel,
                       columns=["Stage","Dataset","Count","% of Stage"])

display(tbl_all)

NameError: name 'raw_orig' is not defined

Interpretation of Table
- Raw rows (ORIG, DL): The original cleaned drillhole records and the deep learning (DL) imputed dataset. 
- XY grid: After snapping coordinates to a regular 10m × 10m grid. The count shows how many unique spatial cells are occupied by each dataset, relative to their raw rows.
- Interval split: Partition of intervals within each grid cell.
	- Overlap: ORIG and DL both have values in the same depth range (used to compare DL predictions vs. real measurements).
	- Orig-only: ORIG has values but DL did not impute (baseline data).
	- DL-only: DL predicted values where ORIG had missing data (new information).
- Voxel split: Final 3D aggregation into 10m × 10m × 1m voxels.
	- Overlap voxels show where ORIG and DL can be directly compared.
	- Orig-only voxels dominate, representing the baseline dataset.
	- DL-only voxels are smaller in number (~2.6%) but represent the incremental insights brought by DL. These are the areas with the highest potential exploration value.

Surface: XY Overlap / Only

In [ ]:
# --- Surface comparison at XY grid cell level (no depth) ---

def surface_xy_grids(df: pd.DataFrame, grid_step: float = GRID_STEP_DEG, agg: str = "mean") -> pd.DataFrame:
    """
    Aggregate surface points into XY grid cells with CU summary and count.
    Returns columns: _gx, _gy, LONc, LATc, CU, COUNT
    """
    D = snap_xy_to_grid(df, step=grid_step)
    agg_df = (
        D.groupby(["_gx","_gy"], as_index=False)
         .agg(CU=("CU", agg), COUNT=("CU", "size"))
    )
    LONc, LATc = grid_cell_centers(agg_df["_gx"].to_numpy(), agg_df["_gy"].to_numpy(), step=grid_step)
    agg_df["LONc"] = LONc; agg_df["LATc"] = LATc
    return agg_df[["_gx","_gy","LONc","LATc","CU","COUNT"]]

def split_surface_overlap(
    sf_orig_grid: pd.DataFrame,
    sf_dl_grid: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Return (overlap_grid, orig_only_grid, dl_only_grid) at grid-cell level.
    For overlap cells, include CU_ORIG, CU_DL, DIFF, DIFF_PCT.
    """
    A = sf_orig_grid.rename(columns={"CU":"CU_ORIG","COUNT":"COUNT_ORIG"})
    B = sf_dl_grid  .rename(columns={"CU":"CU_DL",  "COUNT":"COUNT_DL"})

    M = A.merge(B, on=["_gx","_gy"], how="outer", indicator=True)

    overlap = M.loc[M["_merge"]=="both"].copy()
    overlap["DIFF"] = overlap["CU_DL"] - overlap["CU_ORIG"]
    eps = 1e-9
    denom = overlap["CU_DL"].where(overlap["CU_DL"].abs()>=eps, np.nan)
    overlap["DIFF_PCT"] = 100.0 * overlap["DIFF"] / denom

    orig_only = M.loc[M["_merge"]=="left_only"].copy()
    dl_only   = M.loc[M["_merge"]=="right_only"].copy()

    return overlap, orig_only, dl_only

Run: Drillhole end-to-end

In [ ]:
# --- Run drillhole pipeline end-to-end ---

# 1) Standardize
dh_orig = standardize_drillhole(df_dh_orig_clean)
dh_dl   = standardize_drillhole(df_dh_dl_clean)

print("Drillhole standardized shapes:", dh_orig.shape, dh_dl.shape)

# 2) Split by overlap
dh_overlap, dh_orig_only, dh_dl_only = split_drillhole_overlap(dh_orig, dh_dl)

print("[Drillhole] raw rows:",
      f"overlap pairs={len(dh_overlap):,}, orig-only={len(dh_orig_only):,}, dl-only={len(dh_dl_only):,}")

# 3) Convert intervals -> 3D voxel points
pts_overlap, pts_origonly, pts_dlonly, pts_all, dh_counts = dh_intervals_to_points(
    dh_overlap, dh_orig_only, dh_dl_only
)

print("[Drillhole] voxel counts:",
      f"overlap={dh_counts['vox_overlap']:,}, orig-only={dh_counts['vox_orig_only']:,},",
      f"dl-only={dh_counts['vox_dl_only']:,}, all={dh_counts['vox_all']:,}")

# 4) Export CSVs for Streamlit (3D viewer)
pts_overlap.to_csv(OUT_DIR/"drillhole_points_overlap.csv", index=False)
pts_origonly.to_csv(OUT_DIR/"drillhole_points_origonly.csv", index=False)
pts_dlonly  .to_csv(OUT_DIR/"drillhole_points_dlonly.csv", index=False)
pts_all     .to_csv(OUT_DIR/"drillhole_points_all.csv", index=False)

# Also keep interval-level overlap table (useful for audit)
dh_overlap.to_csv(OUT_DIR/"drillhole_overlaps.csv", index=False)

print("[SAVE] Drillhole CSVs written to", OUT_DIR)

Drillhole standardized shapes: (2464631, 9) (920013, 8)
[Drillhole] raw rows: overlap pairs=260,797, orig-only=2,317,904, dl-only=109,124
[Drillhole] voxel counts: overlap=90,439, orig-only=1,787,661, dl-only=50,084, all=1,926,606
[SAVE] Drillhole CSVs written to ../reports/task2


Run: Surface end-to-end

In [ ]:
# --- Run surface pipeline end-to-end (2D grids) ---

Quick Summary Table

In [ ]:
# --- Summary table for slides: raw vs grid vs split counts ---

,Dataset,Raw rows (orig),Raw rows (dl),Grid voxels (all),Voxels overlap,Voxels orig-only,Voxels dl-only,Grid cells (orig),Grid cells (dl),Cells overlap,Cells orig-only,Cells dl-only
0,Drillhole,2464631,920013,1926606.0,90439.0,1787661.0,50084.0,NaN,NaN,NaN,NaN,NaN
1,Surface,981318,544886,NaN,NaN,NaN,NaN,683953.0,403109.0,48319.0,635634.0,354790.0
